In [148]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
#%run quantexperiment.py
#%load quantexperiment.py

In [149]:
# %load quantexperiment.py
import requests
import os
from functools import wraps
import inspect
import sys
# Pandas became an optional dependency, but we still want to track it
try:
    import pandas
    _PANDAS_FOUND = True
except ImportError:
    _PANDAS_FOUND = False
import csv


class AlphaVantage(object):
    """ Base class where the decorators and base function for the other
    classes of this python wrapper will inherit from.
    """
    _ALPHA_VANTAGE_API_URL = "http://www.alphavantage.co/query?"
    _ALPHA_VANTAGE_MATH_MAP = ['SMA', 'EMA', 'WMA', 'DEMA', 'TEMA', 'TRIMA',
                               'T3', 'KAMA', 'MAMA']
    _ALPHA_VANTAGE_DIGITAL_CURRENCY_LIST = \
        "https://www.alphavantage.co/digital_currency_list/"

    def __init__(self, key=None, output_format='json',
                 treat_info_as_error=True, indexing_type='date', proxy=None):
        """ Initialize the class

        Keyword Arguments:
            key:  Alpha Vantage api key
            retries:  Maximum amount of retries in case of faulty connection or
                server not able to answer the call.
            treat_info_as_error: Treat information from the api as errors
            output_format:  Either 'json', 'pandas' os 'csv'
            indexing_type: Either 'date' to use the default date string given
            by the alpha vantage api call or 'integer' if you just want an
            integer indexing on your dataframe. Only valid, when the
            output_format is 'pandas'
            proxy: Dictionary mapping protocol or protocol and hostname to
            the URL of the proxy.
        """
        if key is None:
            key = os.getenv('ALPHAVANTAGE_API_KEY')
        if not key or not isinstance(key, str):
            raise ValueError('The AlphaVantage API key must be provided '
                             'either through the key parameter or '
                             'through the environment variable '
                             'ALPHAVANTAGE_API_KEY. Get a free key '
                             'from the alphavantage website: '
                             'https://www.alphavantage.co/support/#api-key')
        self.key = key
        self.output_format = output_format
        if self.output_format is 'pandas' and not _PANDAS_FOUND:
            raise ValueError("The pandas library was not found, therefore can "
                             "not be used as an output format, please install "
                             "manually")
        self.treat_info_as_error = treat_info_as_error
        # Not all the calls accept a data type appended at the end, this
        # variable will be overridden by those functions not needing it.
        self._append_type = True
        self.indexing_type = indexing_type
        self.proxy = proxy or {}

    @classmethod
    def _call_api_on_func(cls, func):
        """ Decorator for forming the api call with the arguments of the
        function, it works by taking the arguments given to the function
        and building the url to call the api on it

        Keyword Arguments:
            func:  The function to be decorated
        """

        # Argument Handling
        if sys.version_info[0] < 3:
            # Deprecated since version 3.0
            argspec = inspect.getargspec(func)
        else:
            argspec = inspect.getfullargspec(func)
        try:
            # Assume most of the cases have a mixed between args and named
            # args
            positional_count = len(argspec.args) - len(argspec.defaults)
            defaults = dict(
                zip(argspec.args[positional_count:], argspec.defaults))
        except TypeError:
            if argspec.args:
                # No defaults
                positional_count = len(argspec.args)
                defaults = {}
            elif argspec.defaults:
                # Only defaults
                positional_count = 0
                defaults = argspec.defaults
        # Actual decorating

        @wraps(func)
        def _call_wrapper(self, *args, **kwargs):
            used_kwargs = kwargs.copy()
            # Get the used positional arguments given to the function
            used_kwargs.update(zip(argspec.args[positional_count:],
                                   args[positional_count:]))
            # Update the dictionary to include the default parameters from the
            # function
            used_kwargs.update({k: used_kwargs.get(k, d)
                                for k, d in defaults.items()})
            # Form the base url, the original function called must return
            # the function name defined in the alpha vantage api and the data
            # key for it and for its meta data.
            function_name, data_key, meta_data_key = func(
                self, *args, **kwargs)
            url = "{}function={}".format(AlphaVantage._ALPHA_VANTAGE_API_URL,
                                         function_name)
            for idx, arg_name in enumerate(argspec.args[1:]):
                try:
                    arg_value = args[idx]
                except IndexError:
                    arg_value = used_kwargs[arg_name]
                if 'matype' in arg_name and arg_value:
                    # If the argument name has matype, we gotta map the string
                    # or the integer
                    arg_value = self.map_to_matype(arg_value)
                if arg_value:
                    # Discard argument in the url formation if it was set to
                    # None (in other words, this will call the api with its
                    # internal defined parameter)
                    if isinstance(arg_value, tuple) or isinstance(arg_value, list):
                        # If the argument is given as list, then we have to
                        # format it, you gotta format it nicely
                        arg_value = ','.join(arg_value)
                    url = '{}&{}={}'.format(url, arg_name, arg_value)
            # Allow the output format to be json or csv (supported by
            # alphavantage api). Pandas is simply json converted.
            if 'json' in self.output_format.lower() or 'csv' in self.output_format.lower():
                oformat = self.output_format.lower()
            elif 'pandas' in self.output_format.lower():
                oformat = 'json'
            else:
                raise ValueError("Output format: {} not recognized, only json,"
                                 "pandas and csv are supported".format(
                                     self.output_format.lower()))
            if self._append_type:
                url = '{}&apikey={}&datatype={}'.format(url, self.key, oformat)
            else:
                url = '{}&apikey={}'.format(url, self.key)
            return self._handle_api_call(url), data_key, meta_data_key
        return _call_wrapper

    @classmethod
    def _output_format(cls, func, override=None):
        """ Decorator in charge of giving the output its right format, either
        json or pandas

        Keyword Arguments:
            func:  The function to be decorated
            override:  Override the internal format of the call, default None
        """
        @wraps(func)
        def _format_wrapper(self, *args, **kwargs):
            call_response, data_key, meta_data_key = func(
                self, *args, **kwargs)
            if 'json' in self.output_format.lower() or 'pandas' \
                    in self.output_format.lower():
                data = call_response[data_key]
                #call_response: "TIME_SERIES_INTRADAY"
                #data_key: 'Time Series (15min)'
                if meta_data_key is not None:
                    meta_data = call_response[meta_data_key]
                else:
                    meta_data = None
                # Allow to override the output parameter in the call
                if override is None:
                    output_format = self.output_format.lower()
                elif 'json' or 'pandas' in override.lower():
                    output_format = override.lower()
                # Choose output format
                if output_format == 'json':
                    return data, meta_data
                elif output_format == 'pandas':
                    if isinstance(data, list):
                        # If the call returns a list, then we will append them
                        # in the resulting data frame. If in the future
                        # alphavantage decides to do more with returning arrays
                        # this might become buggy. For now will do the trick.
                        data_array = []
                        for val in data:
                            data_array.append([v for _, v in val.items()])
                        data_pandas = pandas.DataFrame(data_array, columns=[
                            k for k, _ in data[0].items()])
                    else:
                        data_pandas = pandas.DataFrame.from_dict(data,
                                                                 orient='index',
                                                                 dtype=float)
                    data_pandas.index.name = 'date'
                    if 'integer' in self.indexing_type:
                        # Set Date as an actual column so a new numerical index
                        # will be created, but only when specified by the user.
                        data_pandas.reset_index(level=0, inplace=True)
                    return data_pandas, meta_data
            elif 'csv' in self.output_format.lower():
                return call_response, None
            else:
                raise ValueError('Format: {} is not supported'.format(
                    self.output_format))
        return _format_wrapper

    def set_proxy(self, proxy=None):
        """ Set a new proxy configuration

        Keyword Arguments:
            proxy: Dictionary mapping protocol or protocol and hostname to
            the URL of the proxy.
        """
        self.proxy = proxy or {}

    def map_to_matype(self, matype):
        """ Convert to the alpha vantage math type integer. It returns an
        integer correspondent to the type of math to apply to a function. It
        raises ValueError if an integer greater than the supported math types
        is given.

        Keyword Arguments:
            matype:  The math type of the alpha vantage api. It accepts
            integers or a string representing the math type.

                * 0 = Simple Moving Average (SMA),
                * 1 = Exponential Moving Average (EMA),
                * 2 = Weighted Moving Average (WMA),
                * 3 = Double Exponential Moving Average (DEMA),
                * 4 = Triple Exponential Moving Average (TEMA),
                * 5 = Triangular Moving Average (TRIMA),
                * 6 = T3 Moving Average,
                * 7 = Kaufman Adaptive Moving Average (KAMA),
                * 8 = MESA Adaptive Moving Average (MAMA)
        """
        # Check if it is an integer or a string
        try:
            value = int(matype)
            if abs(value) > len(AlphaVantage._ALPHA_VANTAGE_MATH_MAP):
                raise ValueError("The value {} is not supported".format(value))
        except ValueError:
            value = AlphaVantage._ALPHA_VANTAGE_MATH_MAP.index(matype)
        return value

    def _handle_api_call(self, url):
        """ Handle the return call from the  api and return a data and meta_data
        object. It raises a ValueError on problems

        Keyword Arguments:
            url:  The url of the service
            data_key:  The key for getting the data from the jso object
            meta_data_key:  The key for getting the meta data information out
            of the json object
        """
        response = requests.get(url, proxies=self.proxy)
        if 'json' in self.output_format.lower() or 'pandas' in \
                self.output_format.lower():
            json_response = response.json()
            if "Error Message" in json_response:
                raise ValueError(json_response["Error Message"])
            elif "Information" in json_response and self.treat_info_as_error:
                raise ValueError(json_response["Information"])
            elif "Note" in json_response and self.treat_info_as_error:
                raise ValueError(json_response["Note"])
            return json_response
        else:
            csv_response = csv.reader(response.text.splitlines())
            if not csv_response:
                raise ValueError(
                    'Error getting data from the api, no return was given.')
            return csv_response


In [150]:
# %load datapool.py
#from .quantexperiment import AlphaVantage as av

class DataPool(AlphaVantage):

    @AlphaVantage._output_format
    @AlphaVantage._call_api_on_func
    def get_quote_endpoint(self, symbol):
        """ Return the latest price and volume information for a
         security of your choice

        Keyword Arguments:
            symbol:  the symbol for the equity we want to get its data

        """
        _FUNCTION_KEY = "GLOBAL_QUOTE"
        return _FUNCTION_KEY, 'Global Quote', None

In [151]:
# %load finproducts.py
import requests
#from .quantexperiment import AlphaVantage as av

class Stock():
    def __init__(self,symbol,key=None):
        url = "{}function={}".format(AlphaVantage._ALPHA_VANTAGE_API_URL,
                                         'GlOBAL_QUOTE')
        url = '{}&{}={}'.format(url, 'symbol', symbol.upper())
        url = '{}&apikey={}'.format(url, key)
        response = list(requests.get(url).json()['Global Quote'].values())
        self._open, self._high, self._low, self._price,\
        self._volume, self._latestday , self._prevclose , self._change, \
        self._changepercent = response[1:]

    @property
    def open(self):
        return float(self._price)

    @property
    def high(self):
        return float(self._high)

    @property
    def low(self):
        return float(self._low)

    @property
    def price(self):
        return float(self._price)

    @property
    def volume(self):
        return float(self._volume)

    @property
    def latestTradingDay(self):
        return self._latestday

    @property
    def previousclose(self):
        return float(self._prevclose)

    @property
    def change(self):
        return float(self._change)

    @property
    def changePercent(self):
        return float(self._changepercent)


In [405]:
# %load finproducts.py
import requests
import pandas as pd
#from .quantexperiment import AlphaVantage as av
from time import mktime
from datetime import datetime, date

_Y_API = 'https://query2.finance.yahoo.com/v7/finance/options/'

class VanillaOption():
    def __init__(self,symbol,d,m,y,strike,type):
        """
        :param symbol: symbol for the underlying stock
        :param d: e.g. 1,2,3,...,31
        :param m: e.g. 1,2,....,12
        :param y: e.g. 2018,2019
        :param strike: e.g. 100,105
        :param type: e.g. 'call' or 'put'
        """

        self._strike = strike
        self._epoch = int(round(mktime(date(y, m, d).timetuple()) / 86400, 0) * 86400)
        url = _Y_API + symbol.upper() + '?date=' + str(self._epoch)
        self._response = requests.get(url).json()['optionChain']['result']

        if len(self._response) == 0:
            raise LookupError("Could not corresponding information for the symbol.")

        self._expirationDates = [datetime.utcfromtimestamp(i).date() for i in self._response[0]['expirationDates']]
        self._strikes = self._response[0]['strikes']

        # Raise error when could not find information for relevant expiration date or strike price
        if datetime.utcfromtimestamp(self._epoch).date() not in self._expirationDates:
            raise LookupError("Expiration dates are: ", self._expirationDates)
        elif strike not in self._strikes:
            raise LookupError("Strike prices are: ", self._strikes)

        # Raise error when returned value is empty because sometimes there could be only call or put
        self._type = type.lower()
        if self._type == 'calls' and len(self._response[0]['options'][0]['calls']) == 0:
            raise LookupError('No such call exists.')
        if self._type == 'puts' and len(self._response[0]['options'][0]['puts']) == 0:
            raise LookupError('No such put exists.')

    def optionInfo(self):
        """
        :return: return information for the vanilla option
        """
        self._toLookup = self._response[0]['options'][0][self._type]
        for c in self._toLookup:
            if c['strike'] == self._strike:
                found = c
                break
        found['expiration'] = datetime.utcfromtimestamp(found['expiration']).date()
        found['lastTradeDate'] = datetime.utcfromtimestamp(found['lastTradeDate']).date()
        self._found = pd.DataFrame(columns=found.keys())
        self._found.loc[0] = list(found.values())
        return self._found







class Stock():
    def __init__(self,symbol,key=None):
        url = "{}function={}".format(AlphaVantage._ALPHA_VANTAGE_API_URL,
                                         'GlOBAL_QUOTE')
        url = '{}&{}={}'.format(url, 'symbol', symbol.upper())
        url = '{}&apikey={}'.format(url, key)
        response = list(requests.get(url).json()['Global Quote'].values())
        self._open, self._high, self._low, self._price,\
        self._volume, self._latestday , self._prevclose , self._change, \
        self._changepercent = response[1:]

    @property
    def open(self):
        return float(self._price)

    @property
    def high(self):
        return float(self._high)

    @property
    def low(self):
        return float(self._low)

    @property
    def price(self):
        return float(self._price)

    @property
    def volume(self):
        return float(self._volume)

    @property
    def latestTradingDay(self):
        return self._latestday

    @property
    def previousclose(self):
        return float(self._prevclose)

    @property
    def change(self):
        return float(self._change)

    @property
    def changePercent(self):
        return float(self._changepercent)




In [388]:
help(VanillaOption)

Help on class VanillaOption in module __main__:

class VanillaOption(builtins.object)
 |  VanillaOption(symbol, d, m, y, strike, type)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, symbol, d, m, y, strike, type)
 |      :param symbol: symbol for the underlying stock
 |      :param d: e.g. 1,2,3,...,31
 |      :param m: e.g. 1,2,....,12
 |      :param y: e.g. 2018,2019
 |      :param strike: e.g. 100,105
 |      :param type: e.g. 'call' or 'put'
 |  
 |  optionInfo(self)
 |      :return: return information for the vanilla option
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [413]:
option1 = VanillaOption('JPM',21,6,2019,80,'calls')

In [414]:
option1.optionInfo()

,contractSymbol,strike,currency,lastPrice,change,percentChange,volume,openInterest,bid,ask,contractSize,expiration,lastTradeDate,impliedVolatility,inTheMoney
0,JPM190621C00080000,80.0,USD,24.75,0.0,0.0,1,670,24.5,28.1,REGULAR,2019-06-21,2019-02-28,0.496221,True


In [152]:
key='SNYPM900YT9SADEA'

In [389]:
stock = Stock('AAPL',key)

In [391]:
stock.price

186.12

In [154]:
stock.latestTradingDay

'2019-03-15'

In [236]:
from time import mktime
from datetime import datetime, date
epoch = int(round(mktime(date(2019, 6, 21).timetuple())/86400, 0)*86400)

In [237]:
_Y_API = 'https://query2.finance.yahoo.com/v7/finance/options/'

In [238]:
url = _Y_API +'AAPL' + '?date=' + str(epoch)

In [235]:
datetime.utcfromtimestamp(1561075200).date()

datetime.date(2019, 6, 21)

In [239]:
temp = requests.get(url).json()

In [240]:
xxx = [datetime.utcfromtimestamp(i).date() for i in temp['optionChain']['result'][0]['expirationDates']]

In [241]:
calls = temp['optionChain']['result']

In [252]:
datetime.utcfromtimestamp(1561075200).date() not in xxx

False

In [268]:
tolookup = calls[0]['options'][0]['calls']

In [ ]:
st

In [270]:
for c in tolookup:
    if c['strike'] == 100:
        found = c
        break

In [272]:
found['expiration'] = datetime.utcfromtimestamp(found['expiration']).date()

In [312]:
ret = pd.DataFrame(columns=found.keys())

In [302]:
ret=ret.append(list(found.values()))

/anaconda3/lib/python3.7/site-packages/pandas/core/indexes/api.py:107: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)


In [313]:
ret.loc[0]=list(found.values())

In [314]:
ret

,contractSymbol,strike,currency,lastPrice,change,percentChange,volume,openInterest,bid,ask,contractSize,expiration,lastTradeDate,impliedVolatility,inTheMoney
0,AAPL190621C00100000,100.0,USD,87.55,3.150002,3.732229,3,561,86.45,86.8,REGULAR,2019-06-21,1552675278,0.634281,True


In [394]:
import requests
import numpy as np
from bs4 import BeautifulSoup
from scipy.interpolate import interp1d
TREASURY_URL = "http://www.treasury.gov/resource-center/data-chart-center/interest-rates/Pages/TextView.aspx?data=yield"


def riskfree():
    r = requests.get(TREASURY_URL)
    soup = BeautifulSoup(r.text, 'html.parser')

    table = soup.find("table", attrs={'class' : 't-chart'})
    rows= table.find_all('tr')
    lastrow = len(rows)-1
    cells = rows[lastrow].find_all("td")
    date = cells[0].get_text()
    m1 = float(cells[1].get_text())
    m3 = float(cells[2].get_text())
    m6 = float(cells[3].get_text())
    y1 = float(cells[4].get_text())
    y2 = float(cells[5].get_text())
    y3 = float(cells[6].get_text())
    y5 = float(cells[7].get_text())
    y7 = float(cells[8].get_text())
    y10 = float(cells[9].get_text())
    y20 = float(cells[10].get_text())
    y30 = float(cells[11].get_text())

    years = (0, 1/12, 3/12, 6/12, 12/12, 24/12, 36/12, 60/12, 84/12, 120/12, 240/12, 360/12)
    rates = (0, m1/100, m3/100, m6/100, y1/100, y2/100, y3/100, y5/100, y7/100, y10/100, y20/100, y30/100)
    return interp1d(years, rates)


In [400]:
help(interp1d)

Help on class interp1d in module scipy.interpolate.interpolate:

class interp1d(scipy.interpolate.polyint._Interpolator1D)
 |  interp1d(x, y, kind='linear', axis=-1, copy=True, bounds_error=None, fill_value=nan, assume_sorted=False)
 |  
 |  Interpolate a 1-D function.
 |  
 |  `x` and `y` are arrays of values used to approximate some function f:
 |  ``y = f(x)``.  This class returns a function whose call method uses
 |  interpolation to find the value of new points.
 |  
 |  Note that calling `interp1d` with NaNs present in input values results in
 |  undefined behaviour.
 |  
 |  Parameters
 |  ----------
 |  x : (N,) array_like
 |      A 1-D array of real values.
 |  y : (...,N,...) array_like
 |      A N-D array of real values. The length of `y` along the interpolation
 |      axis must be equal to the length of `x`.
 |  kind : str or int, optional
 |      Specifies the kind of interpolation as a string
 |      ('linear', 'nearest', 'zero', 'slinear', 'quadratic', 'cubic',
 |      

ModuleNotFoundError: No module named 'scipy.optimize.zeros.newton'; 'scipy.optimize.zeros' is not a package

In [513]:
help(newton)

Help on function newton in module scipy.optimize.zeros:

newton(func, x0, fprime=None, args=(), tol=1.48e-08, maxiter=50, fprime2=None)
    Find a zero using the Newton-Raphson or secant method.
    
    Find a zero of the function `func` given a nearby starting point `x0`.
    The Newton-Raphson method is used if the derivative `fprime` of `func`
    is provided, otherwise the secant method is used.  If the second order
    derivative `fprime2` of `func` is provided, then Halley's method is used.
    
    Parameters
    ----------
    func : function
        The function whose zero is wanted. It must be a function of a
        single variable of the form f(x,a,b,c...), where a,b,c... are extra
        arguments that can be passed in the `args` parameter.
    x0 : float
        An initial estimate of the zero that should be somewhere near the
        actual zero.
    fprime : function, optional
        The derivative of the function when available and convenient. If it
        is None 

In [419]:
from datetime import date
epoch = int(round(mktime(date(2019, 6, 21).timetuple()) / 86400, 0) * 86400)
dates = datetime.utcfromtimestamp(epoch).date()

In [421]:
(dates-date.today()).days

97

datetime.date(2019, 6, 21)

In [424]:
# %load constants.py

_Y_API = 'https://query2.finance.yahoo.com/v7/finance/options/'
TREASURY_URL = "http://www.treasury.gov/resource-center/data-chart-center/interest-rates/Pages/TextView.aspx?data=yield"

OVERNIGHT_RATE = 0



In [955]:
# %load mathformulas.py
import requests
import numpy as np
from bs4 import BeautifulSoup
#from .constants import TREASURY_URL, OVERNIGHT_RATE
from scipy.interpolate import interp1d
from scipy.stats import norm
from scipy.optimize import fsolve
import math


def riskfree():
    r = requests.get(TREASURY_URL)
    soup = BeautifulSoup(r.text, 'html.parser')

    table = soup.find("table", attrs={'class' : 't-chart'})
    rows= table.find_all('tr')
    lastrow = len(rows)-1
    cells = rows[lastrow].find_all("td")
    date = cells[0].get_text()
    m1 = float(cells[1].get_text())
    m3 = float(cells[2].get_text())
    m6 = float(cells[3].get_text())
    y1 = float(cells[4].get_text())
    y2 = float(cells[5].get_text())
    y3 = float(cells[6].get_text())
    y5 = float(cells[7].get_text())
    y7 = float(cells[8].get_text())
    y10 = float(cells[9].get_text())
    y20 = float(cells[10].get_text())
    y30 = float(cells[11].get_text())

    years = (0, 1/12, 3/12, 6/12, 12/12, 24/12, 36/12, 60/12, 84/12, 120/12, 240/12, 360/12)
    rates = (OVERNIGHT_RATE, m1/100, m3/100, m6/100, y1/100, y2/100, y3/100, y5/100, y7/100, y10/100, y20/100, y30/100)
    return interp1d(years, rates)


class BlackandScholes:
    def __init__(self, S0, K, tau, r, price, op_type, q=0):
        self._S0 = S0
        self._K = K
        self._tau = tau
        self._r = r
        self._q = q
        self._op_price = price
        self._op_type = op_type
        self.implied_vol = self.implied_vol()

    @staticmethod
    def bs_call(S0, K, tau, r, sigma, q=0):
        d1 = (np.log(S0/K)+(r-q+sigma**2/2)*tau)/(sigma*np.sqrt(tau))
        d2 = d1 - sigma*np.sqrt(tau)
        return S0*np.exp(-q*tau)*norm.cdf(d1) - K*np.exp(-r*tau)*norm.cdf(d2)

    @staticmethod
    def bs_put(S0, K, tau, r, sigma, q=0):
        d1 = (np.log(S0 / K) + (r - q + sigma ** 2 / 2) * tau) / (sigma * np.sqrt(tau))
        d2 = d1 - sigma * np.sqrt(tau)
        return K*np.exp(-r*tau)*norm.cdf(-d2) - S0*np.exp(-q*tau)*norm.cdf(-d1)

    def _fprime(self, sigma):
        logSoverK = np.log(self._S0 / self._K)
        n12 = ((self._r + sigma ** 2 / 2) * self._tau)
        numerd1 = logSoverK + n12
        d1 = numerd1 / (sigma * np.sqrt(self._tau))
        return self._S0 * np.sqrt(self._tau) * norm.pdf(d1) * np.exp(-self._r * self._tau)

    def implied_vol(self):
        if self._op_type.lower() == 'calls':
            func = lambda x: (self.bs_call(self._S0, self._K, self._tau, self._r, x, self._q) - self._op_price)
            initial = np.sqrt(2*math.pi/self._tau)*self._op_price/self._S0
            return fsolve(func, 0.1, fprime=self._fprime)[0]
        if self._op_type.lower() == 'puts':
            func = lambda x: (self.bs_put(self._S0, self._K, self._tau, self._r, x, self._q) - self._op_price)
            initial = np.sqrt(2 * math.pi / self._tau) * self._op_price / self._S0
            return fsolve(func, initial, fprime=self._fprime)[0]



























In [956]:
# %load finproducts.py
import requests
import pandas as pd
#from .quantexperiment import AlphaVantage as av
from time import mktime
from datetime import datetime, date
#from .constants import _Y_API
#from .mathformulas import BlackandScholes
#from .mathformulas import riskfree


class VanillaOption:
    def __init__(self, symbol, d, m, y, strike, type):
        """
        :param symbol: e.g. 'AAPL', 'MSFT'
        :param d: e.g. 1,2,3,...,31
        :param m: e.g. 1,2,....,12
        :param y: e.g. 2018,2019
        :param strike: e.g. 100,105
        :param type: e.g. 'call' or 'put'
        """

        self._strike = strike
        self._symbol = symbol
        self._epoch = int(round(mktime(date(y, m, d).timetuple()) / 86400, 0) * 86400)
        self._date = datetime.utcfromtimestamp(self._epoch).date()
        url = _Y_API + symbol.upper() + '?date=' + str(self._epoch)
        self._response = requests.get(url).json()['optionChain']['result']

        if len(self._response) == 0:
            raise LookupError("Could not corresponding information for the symbol.")

        self._expirationDates = [datetime.utcfromtimestamp(i).date() for i in self._response[0]['expirationDates']]
        self._strikes = self._response[0]['strikes']

        # Raise error when could not find information for relevant expiration date or strike price
        if self._date not in self._expirationDates:
            raise LookupError("Expiration dates are: ", self._expirationDates)
        elif strike not in self._strikes:
            raise LookupError("Strike prices are: ", self._strikes)

        # Raise error when returned value is empty because sometimes there could be only call or put
        self._type = type.lower()
        if self._type == 'calls' and len(self._response[0]['options'][0]['calls']) == 0:
            raise LookupError('No such call exists.')
        if self._type == 'puts' and len(self._response[0]['options'][0]['puts']) == 0:
            raise LookupError('No such put exists.')

        toLookup = self._response[0]['options'][0][self._type]
        for c in toLookup:
            if c['strike'] == self._strike:
                found = c
                break
        found['expiration'] = datetime.utcfromtimestamp(found['expiration']).date()
        found['lastTradeDate'] = datetime.utcfromtimestamp(found['lastTradeDate']).date()
        self._option_info = pd.DataFrame(columns=found.keys())
        self._option_info.loc[0] = list(found.values())

    def option_info(self):
        """
        :return: return information for the vanilla option
        """
        return self._option_info

    def BS_Info(self,key=None,q=0):
        stock_info = Stock(self._symbol,key)
        self._stock_price = stock_info.price
        self._tau = (self._date - date.today()).days/365
        self._r = riskfree()(self._tau)
        self.BandS = BlackandScholes(self._stock_price,self._strike,self._tau,self._r,
                                      self._option_info['bid'][0],self._type,q)
        print("price is: ", self.BandS.bs_call(self._stock_price, self._strike, self._tau, self._r, self.BandS.implied_vol, 0))
        return self.BandS.implied_vol



class Stock:
    def __init__(self,symbol,key=None):
        url = "{}function={}".format(AlphaVantage._ALPHA_VANTAGE_API_URL, 'GlOBAL_QUOTE')
        url = '{}&{}={}'.format(url, 'symbol', symbol.upper())
        url = '{}&apikey={}'.format(url, key)
        response = list(requests.get(url).json()['Global Quote'].values())
        self._open, self._high, self._low, self._price,\
        self._volume, self._latestday , self._prevclose , self._change, \
        self._changepercent = response[1:]

    @property
    def open(self):
        return float(self._price)

    @property
    def high(self):
        return float(self._high)

    @property
    def low(self):
        return float(self._low)

    @property
    def price(self):
        return float(self._price)

    @property
    def volume(self):
        return float(self._volume)

    @property
    def latestTradingDay(self):
        return self._latestday

    @property
    def previousclose(self):
        return float(self._prevclose)

    @property
    def change(self):
        return float(self._change)

    @property
    def changePercent(self):
        return float(self._changepercent)




In [960]:
option1 = VanillaOption('AAPL',21,6,2019,160,'calls')

In [961]:
option1.option_info()

,contractSymbol,strike,currency,lastPrice,change,percentChange,volume,openInterest,bid,ask,contractSize,expiration,lastTradeDate,impliedVolatility,inTheMoney
0,AAPL190621C00160000,160.0,USD,28.04,2.210001,8.555946,174,12691,27.9,28.15,REGULAR,2019-06-21,2019-03-15,0.286567,True


In [962]:
option1.BS_Info(key)

price is:  27.899999999999977


0.22252227315418324